In [1]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model

import numpy as np

Using TensorFlow backend.


In [4]:
!ls images/2style2style/A/A1.png''

A  BP


In [ ]:
A_path ='images/2style2style'
img_A = image.load_img(A_path, target_size=(448, 448))

A = image.img_to_array(img_A)
A = np.expand_dims(A, axis=0)
A = preprocess_input(A)

Bp_path = 'img_B_prime_placeholder.jpg'
img_Bp = image.load_img(Bp_path, target_size(448, 448))

Bp = image.img_to_array(img_Bp)
Bp = np.expand_dims(Bp, axis=0)
Bp = preprocess_input(Bp)
vgg19 = VGG19(include_top=False, input_shape=(448, 448, 3))

block1_conv1 = Model(inputs=vgg19.input, output=vgg19.get_layer('block1_conv1').output)
block2_conv1 = Model(inputs=vgg19.input, output=vgg19.get_layer('block2_conv1').output)
block3_conv1 = Model(inputs=vgg19.input, output=vgg19.get_layer('block3_conv1').output)
block4_conv1 = Model(inputs=vgg19.input, output=vgg19.get_layer('block4_conv1').output)
block5_conv1 = Model(inputs=vgg19.input, output=vgg19.get_layer('block5_conv1').output)


F1A = block1_conv1.predict(A)
F2A = block2_conv1.predict(A)
F3A = block3_conv1.predict(A)
F4A = block4_conv1.predict(A)
F5A = block5_conv1.predict(A)

F1Bp = block1_conv1.predict(Bp)
F2Bp = block2_conv1.predict(Bp)
F3Bp = block3_conv1.predict(Bp)
F4Bp = block4_conv1.predict(Bp)
F5Bp = block5_conv1.predict(Bp)

F5Aprime, F5B = F5A, F5Bp